In [39]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv('data_new.csv')

# 6. Обучение модели.

Предсказание цены биткоина - это задача регрессии. Нам нужно предсказать значение целевой переменной в виде вещественного числа. Для оценки качества модели будем использовать традиционные метрики для такого типа задач - это средняя квадратичная ошибка и средняя абсолютная ошибка.

In [41]:
# У нас есть один категориальный признак - workday, нужно его закодировать, это лучше сделать ручками, потому что всего два значения:
df['is_workday'] = df['workday'].apply(lambda x: 1 if x == 'workday' else 0)
df = df.drop(['workday'], axis =1)

df = pd.get_dummies(df, columns=['US President'])
df.replace({True: 1, False: 0}, inplace=True)

In [42]:
# Далее поделим выборку на тестовую и тренировочную

X = df.drop('BTC_price', axis=1)
X = X.drop('Date', axis = 1)
y = df['BTC_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
# Нам нужно отшкалировать признаки, чтобы наша модель лучше работала

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

## 7.1 Обучение линейной регрессии с регуляризацией и без

In [44]:
linear_reg = LinearRegression()
linear_reg.fit(X_train_sc, y_train)
y_train_pred = linear_reg.predict(X_train_sc)
msetrain = mean_squared_error(y_train, y_train_pred)
maetrain = mean_absolute_error(y_train, y_train_pred)
y_test_pred = linear_reg.predict(X_test_sc)
msetest = mean_squared_error(y_test, y_test_pred)
maetest = mean_absolute_error(y_test, y_test_pred)

print("MSE train", msetrain)
print('MAE train', maetrain)
print("MSE test", msetest)
print('MAE test', maetest)

MSE train 29630480.755290076
MAE train 3980.8024611155397
MSE test 26764540.078187756
MAE test 3815.8206638441916


In [45]:
linear_weights = pd.Series(linear_reg.coef_, index=X.columns, name='Linear') # посмотрим на веса признаков - проверим на переобученность модели (хотя результаты метрик на трейне и тесте об этом не свидетельствуют)
print(linear_weights)

Unnamed: 0.1             4177.499365
Unnamed: 0               4177.499365
NASDAQ_price           -30010.289955
S&P500_price            83777.301223
WTI_price                -724.675924
GOLD_price              39111.231821
US Inflation             2231.555010
US Interest Rate         -389.055425
US Unemployment Rate     -875.637126
NASDAQ_sqrt             45634.264175
S&P500_sqrt            -81536.969598
GOLD_sqrt              -39755.589309
is_workday               -670.728315
US President_Biden       2948.665840
US President_Obama       -896.742617
US President_Trump      -2802.439802
Name: Linear, dtype: float64


In [46]:
# Попорбуем добавить Лассо регуляризацию, добавим алгоритм для подбора наиболее подходящего гиперпараметра
alphas = [0.1, 1, 10, 100, 200]
lasso_cv = LassoCV(alphas=alphas, cv=5) # параметр кросс-валидации берем равным 5 чуйке автора
lasso_cv.fit(X_train_sc, y_train)
average_mse = np.mean(lasso_cv.mse_path_, axis=1)
best_alpha_idx = np.argmin(average_mse)
best_alpha = lasso_cv.alphas_[best_alpha_idx]

print("Лучшее значение:", best_alpha)

Лучшее значение: 0.1


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4231002483.23098, tolerance: 30595146.61495093
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13806122211.001993, tolerance: 30595146.61495093
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15434692667.417694, tolerance: 30595146.61495093
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWa

In [47]:

lasso = Lasso(alpha=0.1)
lasso.fit(X_train_sc, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.951e+10, tolerance: 3.805e+07
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [48]:
y_pred_lasso =  lasso.predict(X_test_sc)

mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mae_lasso = mean_absolute_error(y_test,  y_pred_lasso)

print(f"MSE Linear: {msetest}")
print(f"MAE Linear: {maetest}")
print(f"MSE Lasso: {mse_lasso}")
print(f"MAE Lasso: {mae_lasso}")

MSE Linear: 26764540.078187756
MAE Linear: 3815.8206638441916
MSE Lasso: 25791933.1985304
MAE Lasso: 3680.517549571414


## 7.2 Обучение модели kNN

In [49]:


model = KNeighborsRegressor()
neighbors_list = [3, 5, 10, 15, 20, 30, 50] # задаем возможный список оптимального количества соседей


param_grid = {'n_neighbors': neighbors_list}
grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5) #Используем GridSearchCV для подбора оптимального количества соседей
grid_search.fit(X_train_sc, y_train)

optimal_neighbors = grid_search.best_params_['n_neighbors']
optimal_model = KNeighborsRegressor(n_neighbors=optimal_neighbors)
optimal_model.fit(X_train_sc, y_train)

y_pred = optimal_model.predict(X_test_sc)
mse_knn = mean_squared_error(y_test, y_pred) #используем те же самые метрики качества модели для сопоставимости результатов
mae_knn =  mean_absolute_error(y_test, y_pred)
print(f"Оптимальное количество соседей: {optimal_neighbors}")
print(f"MSE: {mse_knn}")
print(f'MAE: {mae_knn}')
# здесь интересно, что модель обученная на нешкалированных данных сильно хуже, той что училась на шкалированных. Я не совсем понимаю почему это работает в кнн.

Оптимальное количество соседей: 3
MSE: 3476748.195651392
MAE: 926.5348811700184


## Добавляем немного деревьев

In [50]:
model = RandomForestRegressor(n_estimators=200, min_impurity_decrease=100, random_state = 42) #рещили попробовать добавить деревья (мы не прооходили их в курсе), гиперпараметры ручками потыкали
model.fit(X_train, y_train)

predictions = model.predict(X_test)
mse_rf = mean_squared_error(y_test, predictions)
mae_rf =  mean_absolute_error(y_test, predictions)
print(mse_rf)
print(mae_rf)

2664087.893831436
815.8355757970049


Мы еще потестили без признаков, на которые наложили квадрат, но это только снизило качество моделей. К сожалению, сюда это не вошло. Тут еще много можно покрутить модель, но не успели. Пишу это в 23:57(((

### Выводы

Как ни странно самую маленьнкую ошибку на тестовой выборке дала модель обученная методом kNN, чуть лучше справляются деревья, а совсем плохо показывает обычная себя линейная регрессия, и даже регуляризация ей не помогла. Результаты немного удивляют - мы делали ставку на последнюю из вышеперечисленных.
Касаемо качества полученных моделей, мы можем ориентироваться на результаты kNN, то есть наша модель ошибается в среднем на 886 долларов по модулю, кажется, модель неприменима в реальной жизни. Вероятнее всего это связано сильной, иногда необъяснимой, волатильностью цены биткоина и скудности нашего датасета.